In [9]:
'''
ГЛОБАЛЬНОЕ ВЫРАВНИВАНИЕ
'''

import numpy as np

sA = 'TTAGCTGATCTTAC' 
sB = 'TTAGGCTATCGA'

def NW(sA, sB):
    n = len(sA)
    m = len(sB)
    sc = np.zeros((m+1, n+1))
    for i in range(0, m + 1):
        sc[i][0] = gpen * i
    for j in range(0, n + 1):
        sc[0][j] = gpen * j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            mis = int(sc[i - 1][j - 1] + mscore(sA[j-1], sB[i-1]))
            d = sc[i - 1][j] + gpen
            ins = sc[i][j - 1] + gpen
            sc[i][j] = max(mis, d, ins)
    alA,alB,i,j = "","",m,n
    while i*j > 0:
        sc_cur,sc_diag,sc_up,sc_left = sc[i][j],sc[i-1][j-1],sc[i][j-1],sc[i-1][j]
        if sc_cur == sc_diag + mscore(sA[j-1], sB[i-1]):
            alA += sA[j-1]
            alB += sB[i-1]
            i -= 1
            j -= 1
        elif sc_cur == sc_up + gpen:
            alA += sA[j-1]
            alB += '-'
            j -= 1
        elif sc_cur == sc_left + gpen:
            alA += '-'
            alB += sB[i-1]
            i -= 1
    while j > 0:
        alA += sA[j-1]
        alB += '-'
        j -= 1
    while i > 0:
        alA += '-'
        alB += sB[i-1]
        i -= 1
    alA = alA[::-1]
    alB = alB[::-1]

    return(alA, alB)
gpen,match,mpen = -1,1,-1


def mscore(a, b):
    if a == b:
        return match
    elif a == '-' or b == '-':
        return gpen
    else:
        return mpen



nsA, nsB = NW(sA, sB)

print(nsA)
print(nsB)





TTA-GCTGATCTTAC
TTAGGCT-ATC-GA-


In [8]:
import numpy 
import argparse
import os
import re
import sys
import unittest
import time
from pandas import *


'''
ЛОКАЛЬНОЕ ВЫРАВНИВАНИЕ
'''


seqA = 'TTAGCTGATCTTAC' 
seqB = 'TTAGGCTATCGA'
match = 3
mismatch = -1
alphabet = 'ACGT'

def main():  
    Substitute = create_Substitute(alphabet, match, mismatch)
    rows = len(seqA) + 1
    cols = len(seqB) + 1
    score_matrix, start_pos, max_score, path_matrix = create_score_matrix(rows, cols)  
    seqA_aligned, seqB_aligned = traceback(path_matrix, start_pos, score_matrix)
    print(seqA_aligned)
    print(seqB_aligned)

    return


def create_score_matrix(rows, cols):
    score_matrix = [[0 for col in range(cols)] for row in range(rows)]
    path_matrix = [[[0 , 'NULL'] for col in range(cols)] for row in range(rows)]  
    
    max_score = 0
    max_pos   = None    
    for i in range(1, rows): 
        for j in range(1, cols):
            score, antecedent = calc_score(score_matrix, i, j)
            if score > max_score:                  
                max_score = score
                max_pos   = (i, j)

            score_matrix[i][j],path_matrix[i][j] = score, antecedent           

    return score_matrix, max_pos, max_score, path_matrix   

def create_Substitute(alphabet, match, mismatch):
    global Substitute 
    Substitute = [['NULL' for col in range(len(alphabet))] for row in range(len(alphabet))] 

    for i in range(len(alphabet)):
        for j in range(len(alphabet)):
            if alphabet[i] == alphabet[j] :
                Substitute[i][j] = match
            elif alphabet[i] != alphabet[j]:
                Substitute[i][j] = mismatch
                
    
    return Substitute


def Substitution_score(Substi, x, y):
        a_i = alphabet.index(seqA[x-1])
    b_j = alphabet.index(seqB[y-1])
    
    return Substitute[a_i][b_j]
    

def traceback(path_matrix, start_pos, score_matrix):
    x,y = start_pos
    aligned_seqA = []
    aligned_seqB = []

    i = len(seqA)
    j = len(seqB)
    
    while x < i :
        aligned_seqA += seqA[i-1]
        aligned_seqB += '-'
        i -= 1
    while y < j:
        aligned_seqA += '-'
        aligned_seqB += seqB[j-1]
        j -= 1
    
    while path_matrix[x][y] != [0, 'NULL'] :
        d, direction = path_matrix[x][y][0], path_matrix[x][y][1]
        if direction == 'DIAG' :
            aligned_seqA.append(seqA[x - 1])     
            aligned_seqB.append(seqB[y - 1])
            x -= 1
            y -= 1
        elif direction == 'UP' : 
            for c in range(d):
                aligned_seqA.append(seqA[x - 1])
                aligned_seqB.append('-')             
                x -= 1
        elif direction == 'LEFT' :
            for c in range(d):
                aligned_seqA.append('-')            
                aligned_seqB.append(seqB[y - 1])
                y -= 1
    
    while x > 0:
        aligned_seqA += seqA[x-1]
        aligned_seqB += '_'
        x -= 1
    while y > 0:
        aligned_seqA += '_'
        aligned_seqB += seqB[y-1]
        y -= 1
    
    return ''.join(reversed(aligned_seqA)), ''.join(reversed(aligned_seqB))


def first_pos_max(list1):
    maxi = max(list1)
    return [i for i, j in enumerate(list1) if j == maxi][0]


def gap_penalty(k) : 
    return 3*k


def calc_score(score_matrix, x, y): 
    similarity = Substitution_score(Substitute, x, y)

    
    same_row = [(score_matrix[x][y-l]-gap_penalty(l)) for l in range(1,x+1)]
    same_col = [(score_matrix[x-k][y]-gap_penalty(k)) for k in range(1,x+1)]
    
    up_score = max(same_col)
    left_score = max(same_row)
    
    
    diag_score = score_matrix[x - 1][y - 1] + similarity
    pos_max_up = first_pos_max(same_col)    
    pos_max_left = first_pos_max(same_row)
                  

    score =  max(0, diag_score, up_score, left_score)  
    
    if score == diag_score :
        antecedent = [1, 'DIAG']
        return score, antecedent
    elif score == up_score :
        antecedent = [pos_max_up + 1, 'UP']
        return score, antecedent
    elif score == left_score : 
        antecedent = [pos_max_left + 1, 'LEFT']
        return score, antecedent 
    else :
        return score, [0, 'NULL']

    
    
main()

TTA-GCTGATC--TTAC
TTAGGCT-ATCGA----
